In [21]:
import os
import csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df1 = pd.read_csv('datasets/players.csv')
df2 = pd.read_csv('datasets/players_teams.csv')
df3 = pd.read_csv('datasets/awards_players.csv')
df4 = pd.read_csv('datasets/teams.csv')
df5 = pd.read_csv('datasets/teams_post.csv')
df6 = pd.read_csv('datasets/coaches.csv')
df7 = pd.read_csv('datasets/series_post.csv')

def corrige_vencedor(teams, series_post):
    # Itera sobre cada rodada ('F', 'CF', 'FR') para ajustar cada fase dos playoffs
    for round_type in ['FR', 'CF', 'F']:
        # Filtra a série específica da rodada
        series_round = series_post[series_post['round'] == round_type]
        
        # Atualiza cada série individualmente
        for _, row in series_round.iterrows():
            year = row['year']
            winner_id = row['tmIDWinner']
            loser_id = row['tmIDLoser']
            
            # Define as colunas que correspondem às rodadas
            if round_type == 'FR':
                round_column = 'firstRound'
            elif round_type == 'CF':
                round_column = 'semis'
            elif round_type == 'F':
                round_column = 'finals'
            
            # Marca o time vencedor como "W" na rodada correspondente
            teams.loc[(teams['year'] == year) & (teams['tmID'] == winner_id), round_column] = 'W'
            
            # Marca o time perdedor como "L" na rodada correspondente
            teams.loc[(teams['year'] == year) & (teams['tmID'] == loser_id), round_column] = 'L'
    
    return teams

teams_file = corrige_vencedor(df4, df7)

players_teams_file = df2.drop(columns=['lgID'])
players_file = df1[df1['pos'].notna() & (df1['pos'] != '')]
players_file = players_file.drop(columns=['firstseason', 'lastseason', 'deathDate', 'collegeOther'])
players_file['college'] = players_file['college'].apply(lambda x: 1 if pd.notnull(x) else 0)
merged_df = pd.merge(players_teams_file, players_file, left_on='playerID', right_on='bioID', how='left')
merged_df = merged_df.drop(columns=['bioID'])
awards_players_file = df3.drop(columns=['lgID'])
teams_file = df4.drop(columns=['lgID', 'divID', 'tmORB','tmDRB','tmTRB','opptmORB','opptmDRB','opptmTRB','seeded'])
teams_file['playoff'] = teams_file['playoff'].apply(lambda x: 1 if x=='Y' else 0)

team_post_file = df5.drop(columns=['lgID'])
series_post_file = df7.drop(columns=['lgIDWinner', 'lgIDLoser'])
coaches_file = df6.drop(columns=['lgID'])


awards_grouped = awards_players_file.groupby(['playerID', 'year'])['award'].apply(list).reset_index()
awards_grouped['award'] = awards_grouped['award'].apply(lambda x: x if isinstance(x, list) else [])

merged_df = pd.merge(merged_df, awards_grouped, on=['playerID', 'year'], how='left')
merged_df['award'] = merged_df['award'].apply(lambda x: x if isinstance(x, list) else [])
merged_df = pd.merge(merged_df, teams_file, on=['tmID','year'], how = 'left')

merged_df = merged_df.drop(columns=['franchID', 'name'])

merged_df = pd.merge(merged_df, team_post_file, on=['tmID','year'], how = 'left')

In [22]:
player_count_per_team_year = merged_df.groupby(['year', 'tmID'])['playerID'].nunique().reset_index()
player_count_per_team_year.columns = ['Year', 'Team', 'PlayerCount']

# print(player_count_per_team_year)

# Calculate min, max, and average player count for each year
summary_stats = player_count_per_team_year.groupby('Year')['PlayerCount'].agg(['min', 'max', 'mean']).reset_index()
summary_stats.columns = ['Year', 'MinPlayerCount', 'MaxPlayerCount', 'AvgPlayerCount']

In [23]:
awards_coaches_file = df3.rename(columns={'playerID': 'coachID'})
coach_awards = awards_coaches_file[awards_coaches_file['award'] == 'Coach of the Year']
coach_awards_grouped = coach_awards.groupby(['coachID', 'year'])['award'].apply(list).reset_index()
coaches_file = pd.merge(coaches_file, coach_awards_grouped, on=['coachID', 'year'], how='left')

In [24]:
avg_oRebounds_by_pos = merged_df.groupby('pos')['oRebounds'].mean().reset_index()
avg_dRebounds_by_pos = merged_df.groupby('pos')['dRebounds'].mean().reset_index()

In [25]:
merged_df = merged_df.drop(columns=['rebounds', 'PostRebounds'])
merged_df = merged_df.rename(columns={'GP_x': 'GP_player', 'GP_y': 'GP_team'})

In [26]:
grouped = merged_df.groupby('year').agg({
    'o_pts': 'sum',
    'o_fga': 'sum',
    'o_oreb': 'sum',
    'o_to': 'sum',
    'o_fta': 'sum',
    'o_asts': 'sum',
    'o_fgm' : 'sum',
    'o_ftm': 'sum',
    'o_dreb':'sum',
}).reset_index()

grouped['VOP'] = grouped['o_pts'] / (grouped['o_fga'] - grouped['o_oreb'] + grouped['o_to'] + 0.44 * grouped['o_fta'])
grouped['factor'] = (2 / 3) - (0.5 * (grouped['o_asts'] / grouped['o_fgm'])) / (2 * (grouped['o_fgm'] / grouped['o_ftm']))
grouped['DRB%'] = (grouped['o_dreb'] - grouped['o_oreb']) / grouped['o_dreb']

uPER_df = merged_df.groupby(['playerID', 'year']).agg({
    'minutes': 'sum',     
    'threeMade': 'sum',   
    'assists': 'sum',     
    'fgMade': 'sum',      
    'ftMade': 'sum',      
    'turnovers': 'sum',   
    'fgAttempted': 'sum', 
    'ftAttempted': 'sum', 
    'dRebounds': 'sum',   
    'oRebounds': 'sum',   
    'steals': 'sum',      
    'blocks': 'sum',      
    'PF': 'sum'           
}).reset_index()

uPER_df = uPER_df.merge(grouped[['year', 'VOP', 'factor', 'DRB%']], on='year')

uPER_df['TRB'] = uPER_df['dRebounds'] + uPER_df['oRebounds']

uPER_df['uPER'] = (1 / uPER_df['minutes']) * (
    uPER_df['threeMade'] +
    (2/3) * uPER_df['assists'] +
    (2 - uPER_df['factor'] * (uPER_df['assists'] / uPER_df['fgMade'])) * uPER_df['fgMade'] +
    (uPER_df['ftMade'] * 0.5 * (1 + (1 - (uPER_df['assists'] / uPER_df['fgMade'])) + (2/3) * (uPER_df['assists'] / uPER_df['fgMade']))) -
    uPER_df['VOP'] * uPER_df['turnovers'] -
    uPER_df['VOP'] * uPER_df['DRB%'] * (uPER_df['fgAttempted'] - uPER_df['fgMade']) -
    uPER_df['VOP'] * 0.44 * (0.44 + (0.56 * uPER_df['DRB%'])) * (uPER_df['ftAttempted'] - uPER_df['ftMade']) +
    uPER_df['VOP'] * (1 - uPER_df['DRB%']) * uPER_df['TRB'] +
    uPER_df['VOP'] * uPER_df['DRB%'] * uPER_df['oRebounds'] +
    uPER_df['VOP'] * uPER_df['steals'] +
    uPER_df['VOP'] * uPER_df['DRB%'] * uPER_df['blocks'] -
    uPER_df['PF'] * ((grouped['o_ftm'].mean() / grouped['o_pts'].mean()) - 0.44 * (grouped['o_fta'].mean() / grouped['o_pts'].mean()) * uPER_df['VOP'])
)

lg_uPER = uPER_df.groupby('year')['uPER'].mean().reset_index()
lg_uPER.rename(columns={'uPER': 'lg_uPER'}, inplace=True)

uPER_df = uPER_df.merge(lg_uPER, on='year')

uPER_df['PER'] = uPER_df['uPER'] * (15 / uPER_df['lg_uPER'])


In [27]:
per_to_merge = uPER_df[['playerID', 'year', 'PER']]
merged_df = merged_df.merge(per_to_merge, on=['playerID', 'year'], how='left')

merged_df['TS%'] = (merged_df['points'] / (2 * (merged_df['fgAttempted'] + 0.44 * merged_df['ftAttempted'])))*100
merged_df['eFG%'] = ((merged_df['fgMade'] + 0.5 * merged_df['threeMade']) / merged_df['fgAttempted'])*100
merged_df['stocks'] = (merged_df['steals'] + merged_df['blocks'])

merged_df['PER'] = merged_df['PER'].fillna(0)
merged_df['TS%'] = merged_df['TS%'].fillna(0)
merged_df['eFG%'] = merged_df['eFG%'].fillna(0)
merged_df['stocks'] = merged_df['stocks'].fillna(0)

#Equipas que não foram aos playoffs
merged_df['W'] = merged_df['W'].fillna(0)
merged_df['L'] = merged_df['L'].fillna(0)

In [28]:
def box_plot_for_each_column(dataset):
    numeric_columns = dataset.select_dtypes(include='number')
    if numeric_columns.empty:
        print("No numeric columns found in the dataset.")
    else:
        numeric_columns.boxplot(figsize=(10, 6))
        plt.title("Boxplot for all numeric columns")
        plt.xticks(rotation=45)  # Rotation in x, if necessary
        plt.show()

def pearson_correlation(dataset, size_x, size_y):
    numeric_columns = dataset.select_dtypes(include='number')
    
    if numeric_columns.empty:
        print("Nenhuma coluna numérica encontrada no dataset.")
    else:
        # Correlation matrix
        correlation_matrix = numeric_columns.corr()

        # View
        plt.figure(figsize=(size_x, size_y))
        sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
        plt.title('Pearson-correlation')
        plt.show()

def bar_chart_for_each_column(dataset):
    non_numeric_columns = dataset.select_dtypes(exclude='number')
    if non_numeric_columns.empty:
        print("Any non-numeric columns found in the dataset.")
    else:
        for column in non_numeric_columns.columns:
            value_counts = non_numeric_columns[column].value_counts()
            plt.figure(figsize=(10, 6))
            value_counts.plot(kind='bar')
            plt.title(f"Bar chart for '{column}'")
            plt.xlabel(column)
            plt.ylabel("Count")
            plt.xticks(rotation=45)
            plt.tight_layout()  # Adjust layout to prevent overlap
            plt.show()

# Pie-chart for each column
def pie_chart_for_each_column(dataset):
    non_numeric_columns = dataset.select_dtypes(exclude='number')
    
    if non_numeric_columns.empty:
        print("Any non-numeric columns found in the dataset.")
    else:
        for column in non_numeric_columns.columns:
            # Count elements from different categories
            category_counts = dataset[column].value_counts()
            
            # Pie-chart
            plt.figure(figsize=(6, 6))
            category_counts.plot.pie(autopct='%1.1f%%', startangle=140)
            plt.title(f'Distribution of {column}')
            plt.ylabel('')  # Remove o rótulo do eixo Y
            plt.show()


#box_plot_for_each_column(merged_df)
#box_plot_for_each_column(coaches_file)
#box_plot_for_each_column(series_post_file)



In [29]:
#pearson_correlation(merged_df, 100, 80)
#pearson_correlation(coaches_file, 8, 6)
#pearson_correlation(series_post_file, 8, 6)

In [30]:
#bar_chart_for_each_column(merged_df)
#bar_chart_for_each_column(coaches_file)
#bar_chart_for_each_column(series_post_file)

In [31]:
#pie_chart_for_each_column(merged_df)
#pie_chart_for_each_column(coaches_file)
#pie_chart_for_each_column(series_post_file)

In [32]:
def replaceGameResults(column):
    return column.apply(lambda value: '100' if value == 'W' else '010' if value == 'L' else '001')

# Aplicar a função para cada coluna específica
merged_df['firstRound'] = replaceGameResults(merged_df['firstRound'])
merged_df['semis'] = replaceGameResults(merged_df['semis'])
merged_df['finals'] = replaceGameResults(merged_df['finals'])


In [33]:
if not os.path.exists('cleanDatasets'):
    os.makedirs('cleanDatasets')

merged_df.to_csv('cleanDatasets/players_and_teams.csv', index=False)
coaches_file.to_csv('cleanDatasets/coaches_and_awards.csv', index=False)
series_post_file.to_csv('cleanDatasets/series_post.csv', index=False)

In [34]:
merged_df2 = merged_df.drop(columns=['minutes','points','threeMade','assists','fgMade','turnovers','fgAttempted','ftAttempted','oRebounds','steals','blocks','PF','o_ftm','o_pts','o_fta','o_pts','o_fga','o_oreb','o_to','o_asts','o_fgm','o_dreb'])
merged_df2 = merged_df2.drop(columns=['GP_player','GS','ftMade','threeAttempted','GP_team'])
merged_df2 = merged_df2.drop(columns=['o_3pm','o_3pa','o_reb','o_pf','o_stl','o_blk','d_fgm','d_fga','d_ftm','d_fta','d_3pm','d_3pa','d_oreb','d_dreb','d_reb','d_asts','d_pf','d_stl','d_to','d_blk','d_pts'])
merged_df2 = merged_df2.drop(columns=['PostGP','PostGS','PostMinutes','PostPoints','PostoRebounds','PostdRebounds','PostAssists','PostSteals','PostBlocks','PostTurnovers','PostPF','PostfgAttempted','PostfgMade','PostftAttempted','PostftMade','PostthreeAttempted','PostthreeMade','PostDQ'])
merged_df2 = merged_df2.drop(columns=['arena'])

merged_df2['birthDate'] = pd.to_datetime(merged_df['birthDate'], errors='coerce').dt.year
merged_df2 = merged_df2.rename(columns={'birthDate': 'birthYear'})

if not os.path.exists('cleanDatasets'):
    os.makedirs('cleanDatasets')

merged_df2.to_csv('cleanDatasets/advancedstatistics.csv', index=False)

In [35]:
players_stats_prevYear = merged_df2[['playerID','year','PER', 'eFG%', 'TS%','stocks','dRebounds']].drop_duplicates().copy()
players_stats_prevYear['year'] = players_stats_prevYear['year'] + 1


players_stats_prevYear = players_stats_prevYear.merge(
    merged_df2[['playerID', 'year', 'tmID', 'playoff']], 
    on=['playerID', 'year'], 
    how='left')

players_stats_prevYear.to_csv('cleanDatasets/players_stats_prevYear.csv', index=False)


In [36]:
#.............................Fazer a media por equipa dos valores mas pegando apenas nos 7 melhores jogadores...............

# Ordenar os jogadores dentro de cada equipe e ano com base no PER (ou outra métrica)
players_stats_prevYear_sorted = players_stats_prevYear.sort_values(by=['tmID', 'year', 'PER'], ascending=[True, True, False])

# Selecionar os 7 melhores jogadores de cada equipe e ano
top_7_players = players_stats_prevYear_sorted.groupby(['tmID', 'year']).head(5)

# Agora, calcular a média de PER, TS%, e eFG% apenas para os 7 melhores jogadores
team_year_stats = top_7_players.groupby(['tmID', 'year', 'playoff'])[['PER', 'TS%', 'eFG%']].mean().reset_index()


# Salva o novo dataset em um arquivo CSV
team_year_stats.to_csv('cleanDatasets/team_year_stats.csv', index=False)

In [37]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

# Selecionar as colunas de interesse
features = team_year_stats[['PER', 'TS%', 'eFG%']]

# Normalizar os dados usando MinMaxScaler
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

# Aplicar o PCA
pca = PCA(n_components=2)
pca.fit(features_scaled)

# Verificar as cargas (coeficientes) dos componentes principais
components = pca.components_

# Baseado nas cargas, você pode decidir as variáveis mais importantes
# Vamos mostrar a importância de cada variável nas componentes principais

# Calcular a soma das cargas absolutas para cada variável
importance = pd.DataFrame(abs(components), columns=['PER', 'TS%', 'eFG%'], index=['PC1', 'PC2'])
importance_sum = importance.sum(axis=0)

# Selecionar as duas variáveis mais importantes
most_important_features = importance_sum.sort_values(ascending=False).head(2)

# Exibir apenas os nomes das variáveis mais importantes
important_variable_names = most_important_features.index.tolist()

# Inicializar a lista de componentes a serem removidos
components_to_drop = ['PER', 'TS%', 'eFG%']

# Remover as variáveis mais importantes da lista de componentes a serem removidos
components_to_drop = [col for col in components_to_drop if col not in important_variable_names]

team_year_stats=team_year_stats.drop(columns=components_to_drop)

# Exibir o resultado
print(team_year_stats)


    tmID  year  playoff        PER       eFG%
0    ATL     9      0.0  19.894077  50.207877
1    ATL    10      1.0  21.751629  51.579845
2    CHA     2      1.0  20.571617  47.843485
3    CHA     3      1.0  19.356376  47.204217
4    CHA     4      1.0  20.989740  46.980211
..   ...   ...      ...        ...        ...
121  WAS     6      0.0  17.387332  46.823234
122  WAS     7      1.0  20.574946  51.330317
123  WAS     8      0.0  21.686321  55.716693
124  WAS     9      0.0  21.568455  45.065012
125  WAS    10      1.0  19.989199  51.201160

[126 rows x 5 columns]


In [38]:
#Dividir o dataset para treino, validacao e teste 

dataset_treino = team_year_stats[(team_year_stats['year'] >= 7) & (team_year_stats['year'] <= 9)]
dataset_teste = team_year_stats[team_year_stats['year'] == 10]

In [39]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neural_network import MLPRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import ElasticNet
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, r2_score

# Features (X) e alvo (y)
X_treino = dataset_treino[important_variable_names] #Antigamente 'PER', 'TS%', 'eFG%'
y_treino = dataset_treino['playoff']

X_teste = dataset_teste[important_variable_names]
y_teste = dataset_teste['playoff']

In [40]:
# Inicializar o modelo de regressão
modelo = RandomForestRegressor(random_state=42)

# Treinar com os dados de treino
modelo.fit(X_treino, y_treino)

y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

print("Mean Squared Error (MSE) na validação:", mse)
print("R² score na validação:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)

Mean Squared Error (MSE) na validação: 0.22048461538461536
R² score na validação: 0.06845250000000025


/tmp/ipykernel_92152/3804687974.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,eFG%,playoff_previsto
1,ATL,10,1.0,21.751629,51.579845,0.81
11,CHI,10,0.0,20.025257,46.367767,0.35
21,CON,10,0.0,22.530104,49.441427,0.55
30,DET,10,1.0,22.996364,44.634029,0.80
47,IND,10,1.0,20.581403,48.096441,0.18
56,LAS,10,1.0,24.575351,47.997875,0.90
67,MIN,10,0.0,23.213399,46.495350,0.65
76,NYL,10,0.0,21.074549,51.153114,0.87
87,PHO,10,1.0,23.627958,51.400423,1.00
98,SAC,10,0.0,20.717580,49.564481,0.50


In [41]:
# Inicializar o modelo de regressão com Gradient Boosting
modelo = GradientBoostingRegressor(random_state=42)

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)


Mean Squared Error (MSE) no teste: 0.17456673434534695
R² score no teste: 0.26245554739090926


/tmp/ipykernel_92152/26995874.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,eFG%,playoff_previsto
1,ATL,10,1.0,21.751629,51.579845,0.916102
11,CHI,10,0.0,20.025257,46.367767,0.216692
21,CON,10,0.0,22.530104,49.441427,0.431419
30,DET,10,1.0,22.996364,44.634029,1.034924
47,IND,10,1.0,20.581403,48.096441,0.372558
56,LAS,10,1.0,24.575351,47.997875,0.855352
67,MIN,10,0.0,23.213399,46.495350,0.671633
76,NYL,10,0.0,21.074549,51.153114,0.807284
87,PHO,10,1.0,23.627958,51.400423,1.016767
98,SAC,10,0.0,20.717580,49.564481,0.627133


In [42]:
# Inicializar o modelo de regressão linear
modelo = LinearRegression()

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)

Mean Squared Error (MSE) no teste: 0.24173684629829584
R² score no teste: -0.021338175610299714


/tmp/ipykernel_92152/2857818035.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,eFG%,playoff_previsto
1,ATL,10,1.0,21.751629,51.579845,0.682466
11,CHI,10,0.0,20.025257,46.367767,0.558427
21,CON,10,0.0,22.530104,49.441427,0.706465
30,DET,10,1.0,22.996364,44.634029,0.695750
47,IND,10,1.0,20.581403,48.096441,0.598739
56,LAS,10,1.0,24.575351,47.997875,0.799216
67,MIN,10,0.0,23.213399,46.495350,0.719924
76,NYL,10,0.0,21.074549,51.153114,0.645325
87,PHO,10,1.0,23.627958,51.400423,0.775701
98,SAC,10,0.0,20.717580,49.564481,0.616042


In [43]:
#::::::::::::::::::::Este modelo é uma variante da regressão linear que usa regularização L2 para reduzir overfitting.::::::::::::::::

# Inicializar o modelo de Ridge Regression
modelo = Ridge(alpha=1.0)  # alpha controla o nível de regularização

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)

Mean Squared Error (MSE) no teste: 0.24159634208347863
R² score no teste: -0.020744545302697093


/tmp/ipykernel_92152/11041507.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,eFG%,playoff_previsto
1,ATL,10,1.0,21.751629,51.579845,0.682479
11,CHI,10,0.0,20.025257,46.367767,0.558429
21,CON,10,0.0,22.530104,49.441427,0.705948
30,DET,10,1.0,22.996364,44.634029,0.694496
47,IND,10,1.0,20.581403,48.096441,0.598750
56,LAS,10,1.0,24.575351,47.997875,0.797805
67,MIN,10,0.0,23.213399,46.495350,0.718815
76,NYL,10,0.0,21.074549,51.153114,0.645526
87,PHO,10,1.0,23.627958,51.400423,0.775030
98,SAC,10,0.0,20.717580,49.564481,0.616180


In [44]:
# Similar ao Ridge, mas utiliza regularização L1. Tende a eliminar variáveis menos importantes, útil para seleção de features.
# Inicializar o modelo de Lasso Regression
modelo = Lasso(alpha=0.1, random_state=42)  # alpha controla a força da regularização

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)


Mean Squared Error (MSE) no teste: 0.2377936128135585
R² score no teste: -0.004678014137284636


/tmp/ipykernel_92152/613287734.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,eFG%,playoff_previsto
1,ATL,10,1.0,21.751629,51.579845,0.643604
11,CHI,10,0.0,20.025257,46.367767,0.569188
21,CON,10,0.0,22.530104,49.441427,0.660157
30,DET,10,1.0,22.996364,44.634029,0.656713
47,IND,10,1.0,20.581403,48.096441,0.593349
56,LAS,10,1.0,24.575351,47.997875,0.719460
67,MIN,10,0.0,23.213399,46.495350,0.670637
76,NYL,10,0.0,21.074549,51.153114,0.620544
87,PHO,10,1.0,23.627958,51.400423,0.702346
98,SAC,10,0.0,20.717580,49.564481,0.603222


In [45]:
# Combina as regularizações L1 (Lasso) e L2 (Ridge).
# Inicializar o modelo de Elastic Net
modelo = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42)
# `alpha` controla a força total da regularização.
# `l1_ratio` controla a proporção de regularização L1 (Lasso) em relação à L2 (Ridge).

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)

Mean Squared Error (MSE) no teste: 0.23917344923616624
R² score no teste: -0.010507823022802176


/tmp/ipykernel_92152/260154350.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,eFG%,playoff_previsto
1,ATL,10,1.0,21.751629,51.579845,0.663061
11,CHI,10,0.0,20.025257,46.367767,0.563809
21,CON,10,0.0,22.530104,49.441427,0.682456
30,DET,10,1.0,22.996364,44.634029,0.674148
47,IND,10,1.0,20.581403,48.096441,0.596063
56,LAS,10,1.0,24.575351,47.997875,0.756998
67,MIN,10,0.0,23.213399,46.495350,0.693440
76,NYL,10,0.0,21.074549,51.153114,0.633273
87,PHO,10,1.0,23.627958,51.400423,0.737915
98,SAC,10,0.0,20.717580,49.564481,0.609863


In [46]:
# Inicializar o modelo de MLP
modelo = MLPRegressor(hidden_layer_sizes=(100, 50),  # Camadas ocultas com 100 e 50 neurônios
                      activation='relu',            # Função de ativação
                      solver='adam',                # Otimizador
                      max_iter=500,                 # Número máximo de iterações
                      random_state=42)

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)


Mean Squared Error (MSE) no teste: 0.23643430118892492
R² score no teste: 0.0010650774767924087


/tmp/ipykernel_92152/2522216952.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,eFG%,playoff_previsto
1,ATL,10,1.0,21.751629,51.579845,0.563412
11,CHI,10,0.0,20.025257,46.367767,0.484648
21,CON,10,0.0,22.530104,49.441427,0.558042
30,DET,10,1.0,22.996364,44.634029,0.567103
47,IND,10,1.0,20.581403,48.096441,0.510536
56,LAS,10,1.0,24.575351,47.997875,0.624006
67,MIN,10,0.0,23.213399,46.495350,0.571658
76,NYL,10,0.0,21.074549,51.153114,0.549271
87,PHO,10,1.0,23.627958,51.400423,0.598470
98,SAC,10,0.0,20.717580,49.564481,0.527694


In [47]:
import numpy as np

# Inicializar o modelo sequencial
modelo = Sequential()

# Adicionar camadas
modelo.add(Dense(64, activation='relu', input_shape=(X_treino.shape[1],)))  # Primeira camada oculta
modelo.add(Dense(32, activation='relu'))  # Segunda camada oculta
modelo.add(Dense(1))  # Camada de saída (regressão)

# Compilar o modelo
modelo.compile(optimizer='adam', loss='mse')

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino, epochs=50, batch_size=32, verbose=1, validation_split=0.2)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste).flatten()  # Flatten para transformar em 1D

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)

/home/emamartins12/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-11-22 18:53:46.589277: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 860ms/step - loss: 8.2904 - val_loss: 6.4122
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 5.2343 - val_loss: 4.1227
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - loss: 3.2494 - val_loss: 2.3883
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 1.8012 - val_loss: 1.2289
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.9059 - val_loss: 0.5592
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.4226 - val_loss: 0.2625
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.2395 - val_loss: 0.2144
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.2716 - val_loss: 0.3215
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.4351 - val_loss: 0.4936
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.6483 - val_loss: 0.6600
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.8406 - val_loss: 0.7724
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.9665 - val_loss: 0.8100

/tmp/ipykernel_92152/1782423946.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,eFG%,playoff_previsto
1,ATL,10,1.0,21.751629,51.579845,0.507618
11,CHI,10,0.0,20.025257,46.367767,0.476967
21,CON,10,0.0,22.530104,49.441427,0.556572
30,DET,10,1.0,22.996364,44.634029,0.612645
47,IND,10,1.0,20.581403,48.096441,0.486453
56,LAS,10,1.0,24.575351,47.997875,0.652073
67,MIN,10,0.0,23.213399,46.495350,0.607909
76,NYL,10,0.0,21.074549,51.153114,0.482938
87,PHO,10,1.0,23.627958,51.400423,0.586673
98,SAC,10,0.0,20.717580,49.564481,0.480600


In [48]:
# Inicializar o modelo de regressão com Extra Trees
modelo = ExtraTreesRegressor(random_state=42)

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)

Mean Squared Error (MSE) no teste: 0.26116923076923076
R² score no teste: -0.10343999999999975


/tmp/ipykernel_92152/1732875705.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,eFG%,playoff_previsto
1,ATL,10,1.0,21.751629,51.579845,0.88
11,CHI,10,0.0,20.025257,46.367767,0.31
21,CON,10,0.0,22.530104,49.441427,0.61
30,DET,10,1.0,22.996364,44.634029,0.75
47,IND,10,1.0,20.581403,48.096441,0.05
56,LAS,10,1.0,24.575351,47.997875,0.97
67,MIN,10,0.0,23.213399,46.495350,0.62
76,NYL,10,0.0,21.074549,51.153114,0.99
87,PHO,10,1.0,23.627958,51.400423,1.00
98,SAC,10,0.0,20.717580,49.564481,0.50


In [49]:
# Inicializar o modelo ElasticNet
modelo = ElasticNet(random_state=42)

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
print(dataset_teste.head(15))


Mean Squared Error (MSE) no teste: 0.23758751658811145
R² score no teste: -0.00380725758477074
    tmID  year  playoff        PER       eFG%  playoff_previsto
1    ATL    10      1.0  21.751629  51.579845          0.585366
11   CHI    10      0.0  20.025257  46.367767          0.585366
21   CON    10      0.0  22.530104  49.441427          0.585366
30   DET    10      1.0  22.996364  44.634029          0.585366
47   IND    10      1.0  20.581403  48.096441          0.585366
56   LAS    10      1.0  24.575351  47.997875          0.585366
67   MIN    10      0.0  23.213399  46.495350          0.585366
76   NYL    10      0.0  21.074549  51.153114          0.585366
87   PHO    10      1.0  23.627958  51.400423          0.585366
98   SAC    10      0.0  20.717580  49.564481          0.585366
105  SAS    10      1.0  19.340974  51.177129          0.585366
114  SEA    10      1.0  19.478406  47.913036          0.585366
125  WAS    10      1.0  19.989199  51.201160          0.585366


/tmp/ipykernel_92152/3962555249.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


In [50]:
# Inicializar o modelo XGBoost Regressor
modelo = xgb.XGBRegressor(random_state=42)

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)


Mean Squared Error (MSE) no teste: 0.21935869408573058
R² score no teste: 0.07320951748778848


/tmp/ipykernel_92152/1187298798.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,eFG%,playoff_previsto
1,ATL,10,1.0,21.751629,51.579845,0.983467
11,CHI,10,0.0,20.025257,46.367767,0.095003
21,CON,10,0.0,22.530104,49.441427,0.327963
30,DET,10,1.0,22.996364,44.634029,0.870367
47,IND,10,1.0,20.581403,48.096441,0.056754
56,LAS,10,1.0,24.575351,47.997875,0.973876
67,MIN,10,0.0,23.213399,46.495350,0.857817
76,NYL,10,0.0,21.074549,51.153114,0.719759
87,PHO,10,1.0,23.627958,51.400423,1.000374
98,SAC,10,0.0,20.717580,49.564481,0.629300


In [51]:
# Inicializar o modelo LightGBM Regressor
modelo = lgb.LGBMRegressor(random_state=42)

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000031 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 30
[LightGBM] [Info] Number of data points in the train set: 41, number of used features: 2
[LightGBM] [Info] Start training from score 0.585366
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


/tmp/ipykernel_92152/3421237157.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,eFG%,playoff_previsto
1,ATL,10,1.0,21.751629,51.579845,0.676715
11,CHI,10,0.0,20.025257,46.367767,0.498367
21,CON,10,0.0,22.530104,49.441427,0.676715
30,DET,10,1.0,22.996364,44.634029,0.587541
47,IND,10,1.0,20.581403,48.096441,0.587541
56,LAS,10,1.0,24.575351,47.997875,0.676715
67,MIN,10,0.0,23.213399,46.495350,0.587541
76,NYL,10,0.0,21.074549,51.153114,0.676715
87,PHO,10,1.0,23.627958,51.400423,0.676715
98,SAC,10,0.0,20.717580,49.564481,0.676715
